# Create source plates for media preparation using biomek

The notebook also generates files with instructions on how to prepare source plates with stock solutions, culture, antiobiotics.

Tested using **ART 3.9.4** kernel on jprime.lbl.gov

## Inputs and outputs

#### Required files to run this notebook:
   
   - `../data/flaviolin/stock_concentrations.csv` 
   

#### Files generated by running this notebook:


   - `24-well_stock_plate_high.csv`, `24-well_stock_plate_low.csv` - instructions on how to prepare the source plates
   
   - `24-well_stock_plate_fresh.csv` - the plate that includes components that need to be prepared fresh for every cycle (culture, FeSO4)
    
The files are stored in the user defined directory. 

## Setup

Importing needed libraries:

In [1]:
import os
import sys
sys.path.append('../')

import pandas as pd
import numpy as np


## User parameters

In [2]:
user_params = {
    'stock_conc_file': '../data/flaviolin/stock_concentrations.csv',
    'output_path': f'../data/flaviolin/',  # Path for output files
}

Load the stock concentrations

In [3]:
df_stock = pd.read_csv(user_params['stock_conc_file'])
df_stock = df_stock.set_index("Component")

In [4]:
df_stock

,Low Concentration[mM],High Concentration[mM],Dilution Factor
Component,,,
MOPS,2000.0,2000.0,1.0
Tricine,400.0,400.0,1.0
H3BO3,0.12,2.4,20.0
Glucose,3000.0,3000.0,1.0
K2SO4,8.7,43.5,5.0
K2HPO4,79.2,396.0,5.0
FeSO4,0.3,6.0,20.0
NH4Cl,1904.0001,1904.0001,1.0
MgCl2,15.6,15.6,1.0


## Create stock solutions plate dataframe


In [5]:
df_stock_plate = pd.DataFrame(
    columns=["Component", "Stock", "Concentration[mM]"])
    
i = 0
num_comp = len(df_stock)
for i in range(2*num_comp):

    if i < num_comp:
        component = df_stock.index[i]
        df_stock_plate.loc[i] = [
            component, 
            "high",
            df_stock.loc[component]["High Concentration[mM]"]
        ]
    elif i < 2*num_comp:
        component = df_stock.index[(i-num_comp)]
        if df_stock.loc[component]["Dilution Factor"] > 1.0:
            df_stock_plate.loc[i] = [
                component, 
                "low",
                df_stock.loc[component]["Low Concentration[mM]"]
            ]
            
df_stock_plate.reset_index(drop=True, inplace=True)

In [6]:
df_stock_plate

,Component,Stock,Concentration[mM]
0,MOPS,high,2000.0
1,Tricine,high,400.0
2,H3BO3,high,2.4
3,Glucose,high,3000.0
4,K2SO4,high,43.5
5,K2HPO4,high,396.0
6,FeSO4,high,6.0
7,NH4Cl,high,1904.0001
8,MgCl2,high,15.6
9,NaCl,high,1500.0


Split the components into two source plates - one with high concentration levels and the other one with low levels

#### High level

In [7]:
df_stock_plate_high = df_stock_plate.copy()
df_stock_plate_high = df_stock_plate_high[df_stock_plate_high['Stock']=='high']
df_stock_plate_high.drop(columns='Stock', inplace=True)


Remove FeSO4 from this plate:

In [8]:
df_stock_plate_high.drop(df_stock_plate_high[df_stock_plate_high['Component']=='FeSO4'].index, inplace=True)


Create additional wells of MgCl2 and NaCl, as those components will be used in larger volumes:

In [9]:
df_stock_plate_high = df_stock_plate_high.append(df_stock_plate_high[df_stock_plate_high['Component']=='MgCl2'], ignore_index=True)
df_stock_plate_high = df_stock_plate_high.append(df_stock_plate_high[df_stock_plate_high['Component']=='NaCl'], ignore_index=True)

In [10]:
df_stock_plate_high

,Component,Concentration[mM]
0,MOPS,2000.0
1,Tricine,400.0
2,H3BO3,2.4
3,Glucose,3000.0
4,K2SO4,43.5
5,K2HPO4,396.0
6,NH4Cl,1904.0001
7,MgCl2,15.6
8,NaCl,1500.0
9,(NH4)6Mo7O24,0.018


Assign well names:

In [11]:
num_source_wells = len(df_stock_plate_high) 

source_well_type = '24-well'
well_rows = 'ABCD'
well_columns = '123456'
    
well_names = [f'{row}{column}' for column in well_columns for row in well_rows]
well_names = well_names[:num_source_wells]

df_stock_plate_high.reset_index(drop=True, inplace=True)
df_stock_plate_high['Well'] = well_names
df_stock_plate_high = df_stock_plate_high.set_index(['Well'])
df_stock_plate_high

,Component,Concentration[mM]
Well,,
A1,MOPS,2000.0
B1,Tricine,400.0
C1,H3BO3,2.4
D1,Glucose,3000.0
A2,K2SO4,43.5
B2,K2HPO4,396.0
C2,NH4Cl,1904.0001
D2,MgCl2,15.6
A3,NaCl,1500.0


#### Low level

In [12]:
df_stock_plate_low = df_stock_plate.copy()
df_stock_plate_low = df_stock_plate_low[df_stock_plate_low['Stock']=='low']
df_stock_plate_low.drop(columns='Stock', inplace=True)
df_stock_plate_low.drop(df_stock_plate_low[df_stock_plate_low['Component']=='FeSO4'].index, inplace=True)
df_stock_plate_low

,Component,Concentration[mM]
16,H3BO3,0.12
17,K2SO4,8.7
18,K2HPO4,79.2
20,(NH4)6Mo7O24,0.0009
21,CoCl2,0.009
22,CuSO4,0.003
23,MnSO4,0.024
24,ZnSO4,0.003


Assign well names:

In [13]:
num_source_wells = len(df_stock_plate_low) 

well_names = [f'{row}{column}' for column in well_columns for row in well_rows]
well_names = well_names[:num_source_wells]

df_stock_plate_low.reset_index(drop=True, inplace=True)
df_stock_plate_low['Well'] = well_names
df_stock_plate_low = df_stock_plate_low.set_index(['Well'])
df_stock_plate_low

,Component,Concentration[mM]
Well,,
A1,H3BO3,0.12
B1,K2SO4,8.7
C1,K2HPO4,79.2
D1,(NH4)6Mo7O24,0.0009
A2,CoCl2,0.009
B2,CuSO4,0.003
C2,MnSO4,0.024
D2,ZnSO4,0.003


#### Fresh stocks plate

Create a plate with fresh stocks of FeSO4 and culture:

In [14]:
fe_low = df_stock_plate[(df_stock_plate['Component']=='FeSO4') & (df_stock_plate['Stock']=='low')]['Concentration[mM]'].values[0]
fe_high = df_stock_plate[(df_stock_plate['Component']=='FeSO4') & (df_stock_plate['Stock']=='high')]['Concentration[mM]'].values[0]

df_stock_plate_fresh = pd.DataFrame(data=[['A1', 'Culture', ''], ['B1', 'FeSO4', fe_low], ['C1', 'FeSO4', fe_high]],
                                    columns=['Well', 'Component', 'Concentration[mM]'], 
                                    )
df_stock_plate_fresh =df_stock_plate_fresh.set_index("Well")
df_stock_plate_fresh

,Component,Concentration[mM]
Well,,
A1,Culture,
B1,FeSO4,0.3
C1,FeSO4,6.0


## Save source plate instructions:

In [15]:
stock_plate_file = f"{user_params['output_path']}/{source_well_type}_stock_plate_high.csv"
df_stock_plate_high.to_csv(stock_plate_file)

In [16]:
stock_plate_file = f"{user_params['output_path']}/{source_well_type}_stock_plate_low.csv"
df_stock_plate_low.to_csv(stock_plate_file)

In [17]:
stock_plate_file = f"{user_params['output_path']}/{source_well_type}_stock_plate_fresh.csv"
df_stock_plate_fresh.to_csv(stock_plate_file)